In [1]:
!pip install google_play_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import time
from google_play_scraper import reviews, Sort
from google_play_scraper.exceptions import GooglePlayScraperException
from tqdm import tqdm

In [4]:
def scrape_reviews(app_id, lang='id', country='id', sort=Sort.MOST_RELEVANT, count=20000, filter_score_with=5):
    """
    Mengambil ulasan aplikasi dari Google Play Store.

    Parameters:
        app_id (str): ID aplikasi di Google Play Store.
        lang (str): Kode bahasa untuk ulasan yang diambil (default: 'id').
        country (str): Kode negara untuk ulasan yang diambil (default: 'id').
        sort (Sort): Metode pengurutan ulasan, misal Sort.MOST_RELEVANT.
        count (int): Jumlah maksimal ulasan yang akan diambil.
        filter_score_with (int): Nilai rating yang difilter, misal hanya ulasan dengan skor 5.

    Returns:
        list: Daftar ulasan yang diambil.
    """
    all_reviews = []
    total_fetched = 0

    # Setup progress bar
    with tqdm(total=count, desc="Mengambil Ulasan", unit="ulasan") as pbar:
        while total_fetched < count:
            try:
                fetched_reviews, _ = reviews(
                    app_id,
                    lang=lang,
                    country=country,
                    sort=sort,
                    count=min(200, count - total_fetched),
                    filter_score_with=filter_score_with
                )
                all_reviews.extend(fetched_reviews)
                total_fetched += len(fetched_reviews)

                # Update progress bar
                pbar.update(len(fetched_reviews))

                # Menghentikan loop jika tidak ada ulasan baru yang ditemukan
                if len(fetched_reviews) == 0:
                    print("Tidak ada ulasan baru yang ditemukan.")
                    break

                # Jeda untuk menghindari pemblokiran
                time.sleep(0.001)

            except GooglePlayScraperException as e:
                print(f"Terjadi kesalahan saat mengambil ulasan: {e}")
                break
            except Exception as e:
                print(f"Kesalahan tidak terduga: {e}")
                break

    return all_reviews

In [5]:
def save_to_csv(data, output_file):
    """
    Menyimpan data ulasan ke dalam file CSV.

    Parameters:
        data (list): Data ulasan yang akan disimpan.
        output_file (str): Nama file output CSV.

    Returns:
        None
    """
    if data:
        df = pd.DataFrame(data)
        df.to_csv(output_file, index=False)
        print(f"Berhasil menyimpan {len(data)} ulasan ke {output_file}")
    else:
        print("Tidak ada data ulasan untuk disimpan.")

In [8]:
def main():
    """
    Fungsi utama untuk mengeksekusi proses scraping dan penyimpanan ulasan.

    Returns:
        None
    """
    app_id = 'com.microsoft.office.officehubrow'
    lang = 'id'
    country = 'id'
    sort = Sort.MOST_RELEVANT
    count = 20000
    filter_score_with = 5
    output_file = 'Ulasan Aplikasi Microsoft Office.csv'

    # Mengambil ulasan aplikasi
    reviews_data = scrape_reviews(app_id, lang, country, sort, count, filter_score_with)

    # Menyimpan ulasan ke dalam file CSV
    save_to_csv(reviews_data, output_file)

In [9]:
# Menjalankan fungsi utama
if __name__ == "__main__":
    main()

Mengambil Ulasan: 100%|██████████| 20000/20000 [00:10<00:00, 1888.83ulasan/s]


Berhasil menyimpan 20000 ulasan ke Ulasan Aplikasi Microsoft Office.csv
